## Setup (Installs, Data, Models)

In [1]:
%pip install llama-index
%pip install llama-index-core==0.10.42
%pip install llama-index-embeddings-openai
%pip install llama-index-postprocessor-flag-embedding-reranker
%pip install git+https://github.com/FlagOpen/FlagEmbedding.git
%pip install llama-index-graph-stores-neo4j
%pip install llama-cloud-services

Note: you may need to restart the kernel to use updated packages.
  Using cached llama_index_core-0.10.42-py3-none-any.whl.metadata (2.4 kB)
Using cached llama_index_core-0.10.42-py3-none-any.whl (15.4 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.14.12
    Uninstalling llama-index-core-0.14.12:
      Successfully uninstalled llama-index-core-0.14.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-cloud-services 0.6.54 requires llama-index-core>=0.12.0, but you have llama-index-core 0.10.42 which is incompatible.
llama-index-indices-managed-llama-cloud 0.9.4 requires llama-index-core<0.15,>=0.13.0, but you have llama-index-core 0.10.42 which is incompatible.
llama-index 0.14.12 requires llama-index-core<0.15.0,>=0.14.12, but you have llama-index-core 0.10.42 which is incompatible.
llama-index-readers-file

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."
os.environ["OPENAI_API_KEY"] = "sk-..."

#### Setup Model

Here we use gpt-4o and default OpenAI embeddings.

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

llm = OpenAI(model="gpt-4o")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

# Best chunk-configuration (https://www.llamaindex.ai/blog/evaluating-the-ideal-chunk-size-for-a-rag-system-using-llamaindex-6207e5d3fec5)
Settings.chunk_size = 1024
Settings.chunk_overlap = 20

print(Settings.context_window)

128000


#### Load Data

Here we load the PDF and parse it with LlamaParse.

In [6]:
from llama_cloud_services import LlamaParse

docs = LlamaParse(result_type="text").load_data("../data/lager.pdf")

Started parsing the file under job_id 66bc49f2-2b99-4d35-bf3f-2ec5236c9c59
.

In [7]:
from copy import deepcopy
from llama_index.core.schema import TextNode, Document
from llama_index.core import VectorStoreIndex


def get_sub_docs(docs):
    """Split docs into pages, by separator."""
    sub_docs = []
    for doc in docs:
        doc_chunks = doc.text.split("\n---\n")
        for doc_chunk in doc_chunks:
            sub_doc = Document(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            sub_docs.append(sub_doc)

    return sub_docs

In [8]:
# this will split into pages
sub_docs = get_sub_docs(docs)

#### Initialize Graph Store

Here we use Neo4j but you can also use our other integrations like Nebula

To launch Neo4j locally, first ensure you have docker installed. Then, you can launch the database with the following docker command

```bash
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4JLABS_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```

From here, you can open the db at [http://localhost:7474/](http://localhost:7474/). On this page, you will be asked to sign in. Use the default username/password of `neo4j` and `neo4j`.

Once you login for the first time, you will be asked to change the password.

After this, you are ready to create your first property graph!

In [9]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

graph_store = Neo4jPGStore(
    username="neo4j",
    password="graph1312",
    url="bolt://localhost:7687",
)
vec_store = None

## Construct Knowledge Graph, Get Retrievers

This section shows you how to construct the knowledge graph over the existing documents.

**Note**: we have the default extractors (implicit path, simple llm path) configured. You can also choose to use a pre-defined schema as mentioned in this [notebook](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/property_graph/property_graph_advanced.ipynb).

In [10]:
from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor,
)
from llama_index.core import PropertyGraphIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

In [25]:
index = PropertyGraphIndex.from_documents(
    sub_docs,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 1/1 [00:04<00:00,  4.01s/it]


## Neo4j

To see the entire graph -> run cypher command ```match n=() return n```

To delete the entire graph -> run cypher command ```match n=() detach delete n```

# Retrieving

## Build Baseline Vector Index

We also build a "baseline" vector index. This follows the "naive" RAG pipeline approach of chunking and vector embedding. We use this as a comparison point.

In [49]:
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine

base_index = VectorStoreIndex.from_documents(sub_docs, embed_model=embed_model)
base_retriever = base_index.as_retriever(similarity_top_k=10)
base_query_engine = RetrieverQueryEngine(base_retriever)

In [14]:
response = base_query_engine.query(
    "Worum geht es in dem Dokument? Antworte in 2-3 Sätzen."
)
print(str(response))

Das Dokument behandelt technische Zeichnungen und die Spezifikationen von Wälzlagern nach verschiedenen DIN-Normen. Es enthält Tabellen mit Maßen und Details zu verschiedenen Lagerarten wie Rillenkugellager, Axialrillenkugellager, Zylinderrollenlager und Kegelrollenlager.


In [15]:
response = base_query_engine.query(
    "Wie ist die Seitenwirkung von dem Axialrillenkugellager?"
)
print(str(response))

Das Axialrillenkugellager ist einseitig wirkend.


In [16]:
response = base_query_engine.query(
    "Was bedeutet 'Das Axialrillenkugellager ist einseitig wirkend'? Antworte knapp basierend auf technische Quellen."
)
print(str(response))

Ein einseitig wirkendes Axialrillenkugellager kann nur axiale Kräfte in eine Richtung aufnehmen.


In [18]:
response = base_query_engine.query(
    "Welches DIN-Bezeichnungen (und Lagerreihen falls vorhanden) sind im Dokument zu finden? Kurz auflisten bitte."
)
print(str(response))

Die im Dokument zu findenden DIN-Bezeichnungen und Lagerreihen sind:

1. Rillenkugellager: DIN 625, Lagerreihe 62
2. Axialrillenkugellager: DIN 711, Lagerreihe 512
3. Zylinderrollenlager: DIN 5412-1, Lagerreihe NU 2
4. Kegelrollenlager: DIN 720, Lagerreihe 302


In [19]:
response = base_query_engine.query(
    "Was bedeutet 'Lagerreihe 62'? Kurz."
)
print(str(response))

'Lagerreihe 62' bezieht sich auf eine spezifische Serie von Rillenkugellagern, die durch bestimmte Maße und Eigenschaften gekennzeichnet ist.


### Test results 

✅ Succeeded all tests

In [23]:
# positive test case
response = base_query_engine.query(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=55"
)
print(str(response))

Die ganze Reihe für den Zylinderrollenlager DIN 5412 mit d=55 ist NU 211E. Die Maße sind D=100, B=21, und r_min=1,1.


### Test results 

✅ Succeeded positive test case

In [38]:
# negative test case
response = base_query_engine.query(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=15"
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=20"
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=85"
)
print(str(response))

Die ganze Reihe für den Zylinderrollenlager DIN 5412 mit den angegebenen Durchmessern ist wie folgt:

- Für d=20: NU 204E mit D=47, B=14, r_min=0,6
- Für d=85: NU 209E mit D=85, B=19, r_min=1,1


### Test results 

✅ Succeeded test case '...mit d=15' <br>
✅ Succeeded test case '...mit d=20' <br>
❌ Failed test case '...mit d=85'
-> Pipeline confused small 'd' with 'D'

In [20]:
print(len(response.source_nodes))
for node in response.source_nodes:
    print("---")
    print(node.get_content())

3
---
Maße von Wälzlagern nach DIN (Auswahl)

                                       C
                                       rₐ    rₐ

rᵃ

rⁱ
                                       ri

                                             B  d D

                                                                                         T

 Rillenkugellager    Axialrillenkugellager    Zylinderrollenlager               Kegelrollenlager
     DIN 625                DIN 711                DIN 5412-1                        DIN 720
                       einseitig wirkend
 Rillenkugellager DIN 625-1                                               Axialrillenkugellager DIN 711
              Lagerreihe 62                                                       Lagerreihe 512
  Kurz-     d  D               B       r      Kurz-                 d     d     D         H                r
 zeichen                              min    zeichen                   w     g  g                         min
   6204     20    

## Build Custom Retriever

Build joint retriever that combines vector and KG search.

#### Define Vector Retriever

Here we define our vector context retriever - it returns initial nodes via vector search, and traverses the relations to pull in more nodes/context.


In [50]:
from llama_index.core.indices.property_graph import VectorContextRetriever

kg_retriever = VectorContextRetriever(
    index.property_graph_store,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    similarity_top_k=7,
    path_depth=1,
    # include_text=False,
    include_text=True,
)

nodes = kg_retriever.retrieve(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=55"
)
# nodes = kg_retriever.retrieve('san francisco')
print(len(nodes))
for idx, node in enumerate(nodes):
    print(f">> IDX: {idx}, {node.get_content()}")

2
>> IDX: 0, Here are some facts extracted from the provided text:

Zylinderrollenlager -> Din 5412-1 -> H
Axialrillenkugellager -> Din 711 -> Lagerreihe 512
Axialrillenkugellager -> Din 711 -> Lagerreihe 512
Rillenkugellager -> Din 625 -> Einseitig wirkend
Kegelrollenlager -> Din 720 -> Min
Wälzlagern -> Maße von -> Din
Wälzlagern -> Auswahl -> Din

Maße von Wälzlagern nach DIN (Auswahl)

                                       C
                                       rₐ    rₐ

rᵃ

rⁱ
                                       ri

                                             B  d D

                                                                                         T

 Rillenkugellager    Axialrillenkugellager    Zylinderrollenlager               Kegelrollenlager
     DIN 625                DIN 711                DIN 5412-1                        DIN 720
                       einseitig wirkend
 Rillenkugellager DIN 625-1                                               Axialrillenkugell

In [51]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both KG vector search and direct vector search."""

    def __init__(self, kg_retriever, vector_retriever):
        self._kg_retriever = kg_retriever
        self._vector_retriever = vector_retriever

    def _retrieve(self, query_bundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""
        kg_nodes = self._kg_retriever.retrieve(query_bundle)
        vector_nodes = self._vector_retriever.retrieve(query_bundle)

        unique_nodes = {n.node_id: n for n in kg_nodes}
        unique_nodes.update({n.node_id: n for n in vector_nodes})
        return list(unique_nodes.values())

In [52]:
custom_retriever = CustomRetriever(kg_retriever, base_retriever)

In [53]:
nodes = custom_retriever.retrieve(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=55"
)
print(len(nodes))
for idx, node in enumerate(nodes):
    print(f">> IDX: {idx}, {node.get_content()}")

5
>> IDX: 0, Here are some facts extracted from the provided text:

Zylinderrollenlager -> Din 5412-1 -> H
Axialrillenkugellager -> Din 711 -> Lagerreihe 512
Axialrillenkugellager -> Din 711 -> Lagerreihe 512
Rillenkugellager -> Din 625 -> Einseitig wirkend
Kegelrollenlager -> Din 720 -> Min
Wälzlagern -> Maße von -> Din
Wälzlagern -> Auswahl -> Din

Maße von Wälzlagern nach DIN (Auswahl)

                                       C
                                       rₐ    rₐ

rᵃ

rⁱ
                                       ri

                                             B  d D

                                                                                         T

 Rillenkugellager    Axialrillenkugellager    Zylinderrollenlager               Kegelrollenlager
     DIN 625                DIN 711                DIN 5412-1                        DIN 720
                       einseitig wirkend
 Rillenkugellager DIN 625-1                                               Axialrillenkugell

## Build Agent

Now that we have the retriever, we can treat it as a RAG pipeline tool, and wrap it with an agent that can perform basic CoT reasoning and maintain conversation memory over time.

In [30]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RetrieverQueryEngine

kg_query_engine = RetrieverQueryEngine(custom_retriever)
kg_query_tool = QueryEngineTool(
    query_engine=kg_query_engine,
    metadata=ToolMetadata(
        name="query_tool",
        description="Provides information about row table lookups",
    ),
)

In [31]:
# from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context

agent = ReActAgent(
    tools=[kg_query_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)

# context to hold this session/state
ctx = Context(agent)

## Try out Queries

Now that the agent is setup, let's try out some queries.

In [65]:
handler = agent.run(
    "Welche Lagerrtypen haben ein Lager mit d=20?",
    ctx=ctx)

In [66]:
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I need to use a tool to find the types of bearings that have an inner diameter of 20.
Action: query_tool
Action Input: {'input': 'Lagertypen mit d=20'}
Observation: The following types of bearings have an inner diameter (d) of 20:

1. Zylinderrollenlager (Cylindrical Roller Bearing): NU 204E
2. Rillenkugellager (Deep Groove Ball Bearing): 6204
3. Kegelrollenlager (Tapered Roller Bearing): 30204
4. Axialrillenkugellager (Thrust Ball Bearing): 51204
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Die folgenden Lagertypen haben einen Innendurchmesser (d) von 20:

1. Zylinderrollenlager: NU 204E
2. Rillenkugellager: 6204
3. Kegelrollenlager: 30204
4. Axialrillenkugellager: 51204Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Die Lagertypen mit einem Innendurchmesser (\(d = 20\)) sind:

1. Rillenkugellager: 6204
2. Axialrillenkugellage

In [67]:
handler = agent.run(
    "Welche Lagerrtypen haben ein Lager mit d=15?",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I need to use a tool to find the types of bearings that have an inner diameter of 15.
Action: query_tool
Action Input: {'input': 'Lagertypen mit d=15'}
Observation: Die Lagertypen mit einem Durchmesser \( d = 15 \) sind:

1. Rillenkugellager: 6202
2. Axialrillenkugellager: 51202
3. Kegelrollenlager: 30202
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Die Lagertypen mit einem Innendurchmesser (\(d = 15\)) sind:

1. Rillenkugellager: 6202
2. Axialrillenkugellager: 51202
3. Kegelrollenlager: 30202Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Es gibt keine Lagertypen mit einem Innendurchmesser (\(d = 15\)) in den angegebenen Daten.

### Test results 

✅ Succeeded postive test case '...mit d=20' <br>
✅ Succeeded negative test case '...mit d=15' <br>

In [ ]:
# positive test case
handler = agent.run(
    "Listen Sie mir bitte die komplette Reihe für den Axialrillenkugellager DIN 711 wo d_w=30",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I need to use a tool to help me answer the question.
Action: query_tool
Action Input: {"input": "komplette Reihe für Axialrillenkugellager DIN 711 mit d_w=30"}
Observation: For an Axialrillenkugellager according to DIN 711 with \( d_w = 30 \), the complete series is 51206. The dimensions are: inner diameter \( d = 30 \), outer diameter \( D = 52 \), height \( H = 16 \), and minimum radius \( r_{\text{min}} = 0.6 \).
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Für ein Axialrillenkugellager nach DIN 711 mit einem Wellendurchmesser (\(d_w\)) von 30 ist die komplette Serie 51206. Die Abmessungen sind: Innendurchmesser \(d = 30\), Außendurchmesser \(D = 52\), Höhe \(H = 16\) und Mindestradius \(r_{\text{min}} = 0,6\).Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Für ein Axialrillenkugellager nach DIN 711 mit einem Wellendurchmess

### Test results 

✅ Succeeded test partially: <br>
    - detected d_w correctly <br>
    - forgot to mention d_g for that row (Parsing could be struggling with subscripts)

In [ ]:
# positive test case
handler = agent.run(
    "Listen Sie mir bitte die komplette(n) Reihe(n) für den Kegelrollenlager DIN 720 wo ri_min=1.5",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I need to use a tool to find the complete series for the specified tapered roller bearing with the given minimum inner radius.
Action: query_tool
Action Input: {'input': 'Kegelrollenlager DIN 720 mit ri_min=1.5'}
Observation: Die komplette Reihe für Kegelrollenlager nach DIN 720 mit einem minimalen \( r_i \) von 1,5 umfasst die folgenden Lager:

- 32007
- 32008
- 32009
- 32010
- 32011
- 32012
- 32013
- 32014
- 32015
- 32016
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Die komplette Reihe für Kegelrollenlager nach DIN 720 mit einem minimalen \( r_i \) von 1,5 umfasst die folgenden Lager:

- 32007
- 32008
- 32009
- 32010
- 32011
- 32012
- 32013
- 32014
- 32015
- 32016Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Die komplette Reihe für Kegelrollenlager nach DIN 720 mit einem minimalen Innenradius (\(ri_{min}\)) von 1,5 umfasst 

### Test results 

❌ Failed: Should have generated rows from 30207, 30208, 30209 abd 30210 only. <br>
Cause: Parsing could be struggling with subscripts

In [ ]:
# negative test case
handler = agent.run(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=15"
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=20"
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=85",
    ctx=ctx)

from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I need to use a tool to find the complete series for cylindrical roller bearings according to DIN 5412 with the specified diameters.
Action: query_tool
Action Input: {'input': 'Zylinderrollenlager DIN5412 mit d=15, d=20, d=85'}
Observation: For Zylinderrollenlager according to DIN 5412, the series for the specified diameters are as follows:

- For \( d = 15 \): The corresponding bearing is NU 203E with \( D = 40 \), \( B = 12 \), and \( r_{\text{min}} = 0.6 \).
- For \( d = 20 \): The corresponding bearing is NU 304E with \( D = 52 \), \( B = 15 \), and \( r_{\text{min}} = 1 \).
- For \( d = 85 \): The corresponding bearing is NU 2217E with \( D = 150 \), \( B = 36 \), and \( r_{\text{min}} = 2.1 \).

Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Für Zylinderrollenlager nach DIN 5412 sind die Reihen für die angegebenen Durchmesser wie folgt:

- Für \(d = 15\): Das entsprechende L

### Test results 

✅  This same test (which failed with basic RAG), succeeded with agentic RAG thanks to the LRM model which is able to launch tools multiple times and reason about whether its answer was correct or not. <br>
<br>
See that the first answer involved some hallucinations or 'smooth guessing'. It guessed the row header 'NU203E' when asked about d=15 although it is not present in the table. It copied the pattern used in the real table, NU204E, then NU205E ... until NU216E, so the generated row header 'NU203' is **very likely** to be true in the sequence. <br>
This is a common mistake that LLMs do, because they are originally not trained to answer in a deterministic way, but rather to be next-token-generators, which sample the token with the highest probability.

✅**BUT** thanks to the thinking mode being enabled by the agent, it was able to identify that row 'd=15' and 'd=85' are NOT present in the table. Only row 'd=20' is present with 'Kurz-zeichen=NU204E'


# Optimizing parsing output ...

In [68]:
# Better parsing
from llama_cloud_services import LlamaParse

parser = LlamaParse(
    parse_mode="parse_page_with_agent",
    model="openai-gpt-4-1-mini",
    high_res_ocr=True,
    adaptive_long_table=True,
    outlined_table_extraction=True,
    output_tables_as_HTML=True,
).load_data("../data/lager.pdf")


from copy import deepcopy
from llama_index.core.schema import TextNode, Document
from llama_index.core import VectorStoreIndex


def get_sub_docs(docs):
    """Split docs into pages, by separator."""
    sub_docs = []
    for doc in docs:
        doc_chunks = doc.text.split("\n---\n")
        for doc_chunk in doc_chunks:
            sub_doc = Document(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            sub_docs.append(sub_doc)

    return sub_docs

# this will split into pages
sub_docs = get_sub_docs(docs)

from llama_index.graph_stores.neo4j import Neo4jPGStore

graph_store = Neo4jPGStore(
    username="neo4j",
    password="graph1312",
    url="bolt://localhost:7687",
)
vec_store = None

from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor,
)
from llama_index.core import PropertyGraphIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

index = PropertyGraphIndex.from_documents(
    sub_docs,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
            num_workers=4,
            max_paths_per_chunk=10,
        ),
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Started parsing the file under job_id 56460a13-c3e7-4c9e-8eb9-83f1056b9ed0


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


## Retrieve ...

In [74]:
# vector index
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine

base_index = VectorStoreIndex.from_documents(sub_docs, embed_model=embed_model)
base_retriever = base_index.as_retriever(similarity_top_k=10)
base_query_engine = RetrieverQueryEngine(base_retriever)

# kg index
from llama_index.core.indices.property_graph import VectorContextRetriever

kg_retriever = VectorContextRetriever(
    index.property_graph_store,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    similarity_top_k=5,
    path_depth=1,
    # include_text=False,
    include_text=True,
)

nodes = kg_retriever.retrieve(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=55"
)

# Custom retriever
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both KG vector search and direct vector search."""

    def __init__(self, kg_retriever, vector_retriever):
        self._kg_retriever = kg_retriever
        self._vector_retriever = vector_retriever

    def _retrieve(self, query_bundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""
        kg_nodes = self._kg_retriever.retrieve(query_bundle)
        vector_nodes = self._vector_retriever.retrieve(query_bundle)

        unique_nodes = {n.node_id: n for n in kg_nodes}
        unique_nodes.update({n.node_id: n for n in vector_nodes})
        return list(unique_nodes.values())
custom_retriever = CustomRetriever(kg_retriever, base_retriever)

nodes = custom_retriever.retrieve(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=55"
)

### New extracted data

In [75]:
print(len(nodes))
for idx, node in enumerate(nodes):
    print(f">> IDX: {idx}, {node.get_content()}")

5
>> IDX: 0, Here are some facts extracted from the provided text:

Zylinderrollenlager -> Din -> 5412-1
Axialrillenkugellager -> Din -> 711
Rillenkugellager -> Din -> 625
Kegelrollenlager -> Din -> 720

Maße von Wälzlagern nach DIN (Auswahl)

                                       C
                                       rₐ    rₐ

rᵃ

rⁱ
                                       ri

                                             B  d D

                                                                                         T

 Rillenkugellager    Axialrillenkugellager    Zylinderrollenlager               Kegelrollenlager
     DIN 625                DIN 711                DIN 5412-1                        DIN 720
                       einseitig wirkend
 Rillenkugellager DIN 625-1                                               Axialrillenkugellager DIN 711
              Lagerreihe 62                                                       Lagerreihe 512
  Kurz-     d  D               B       

## Build new agent

In [76]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RetrieverQueryEngine

kg_query_engine = RetrieverQueryEngine(custom_retriever)
kg_query_tool = QueryEngineTool(
    query_engine=kg_query_engine,
    metadata=ToolMetadata(
        name="query_tool",
        description="Provides information about row table lookups",
    ),
)

# from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context

agent = ReActAgent(
    tools=[kg_query_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False,
)

# context to hold this session/state
ctx = Context(agent)


## Queries

In [80]:
# positive test case
handler = agent.run(
    "Listen Sie mir bitte die komplette Reihe (mit allen Spaltenwerten) für den Axialrillenkugellager DIN 711 wo d_w=30",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I need to use a tool to help me answer the question with all column values for the specified bearing.
Action: query_tool
Action Input: {'input': 'komplette Reihe für Axialrillenkugellager DIN 711 mit d_w=30'}
Observation: For an Axialrillenkugellager according to DIN 711 with \( d_w = 30 \), the complete row includes the following values:

- Bearing Designation: 51206
- Inner Diameter (d): 30 mm
- Outer Diameter (D): 52 mm
- Height (H): 16 mm
- Minimum Radius (r): 0.6 mm
- Dynamic Load Rating (C): 28.1 kN
- Static Load Rating (C0): 52 kN
- Fatigue Load Limit (Pu): 2.1 kN
- Reference Speed: 5300 rpm
- Limiting Speed: 8000 rpm

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Für ein Axialrillenkugellager nach DIN 711 mit \( d_w = 30 \) umfasst die komplette Reihe folgende Werte:

- Lagerbezeichnung: 51206
- Innendurchmesser (d): 30 mm
- Außendurchmesser (D): 52 mm
- Höhe (H): 16 mm
- 

### Test results 
Better test result: <br>
✅ Now the answer includes ALL rows columns and its not missing d_g like before
❌ BUT, the parser (see results at section 'New extracted data') is still struggling by confusing d and d_w, and d_w and d_g (only the symbols) <br>

TO BE HONEST, this is the most complex layout in our files (complex table, multiple table in the same parent table, multiple rows and columns, sub- and superscripts), but the pipeline STILL managed to return correct results.

In [ ]:
# memory test 1
handler = agent.run(
    "I deleted our conversation by mistake, what was my last question about? answer briefly",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: English. I can answer without using any tools.
Answer: Your last question was about listing the complete series for Axialrillenkugellager DIN 711 with \( d_w = 30 \), including all column values.

In [ ]:
# memory test 2
handler = agent.run(
    "And what was your answer to my last question? answer briefly",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: English. I can answer without using any tools.
Answer: My answer was that for Axialrillenkugellager DIN 711 with \( d_w = 30 \), the complete series is 51206, with dimensions \( d = 30 \), \( d_w = 32 \), \( D = 52 \), \( H = 16 \), and \( r_{\text{min}} = 0.6 \).

Test 1 -> Succeeded partially ✅
- Memory is ON
- The extracted values are correct
- BUT the header names are sometimes false (d instead of d_w && d_w instead of d_g)
- model infering H for Höhe, which is false, it should be "Breite" ... 

-> Table structure/layout is very difficult, parse model still struggles with subscripts 

In [84]:
# image test 2
handler = agent.run(
    "Can you briefly describe the figure of the 'Kegelrollenlager DIN 720' in 2-3 sentences in German?",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I can answer without using any tools.
Answer: Das 'Kegelrollenlager DIN 720' hat Innen- und Außenringe mit kegelförmigen Laufbahnen. Die Rollen sind ebenfalls kegelförmig, was es ihnen ermöglicht, sowohl radiale als auch axiale Lasten zu tragen. Dieses Design unterstützt hohe Tragfähigkeiten und ist ideal für Anwendungen, die Langlebigkeit und Präzision erfordern.

Test 2 -> Succeeded totally ✅✅✅

In [78]:
# positive test case
handler = agent.run(
    "Listen Sie mir bitte die komplette(n) Reihe(n) für den Kegelrollenlager DIN 720 wo ri_min=1.5",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I need to use a tool to help me answer the question.
Action: query_tool
Action Input: {"input": "Kegelrollenlager DIN 720 mit ri_min=1.5"}
Observation: For a Kegelrollenlager according to DIN 720 with \( r_{i,\text{min}} = 1.5 \), the corresponding bearing designations are 30203, 30303, 31303, 32203, 32303, and 33203.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Für ein Kegelrollenlager nach DIN 720 mit \( r_{i,\text{min}} = 1.5 \) lauten die entsprechenden Lagerbezeichnungen 30203, 30303, 31303, 32203, 32303 und 33203.Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Für ein Kegelrollenlager nach DIN 720 mit einem minimalen Innenradius (ri_min) von 1,5 sind die geeigneten Optionen:

- 30207
- 30208
- 30209
- 30210

Diese Lager haben den angegebenen minimalen Innenradius von 1,5.

Test 3 -> Succeeded totally ✅✅✅

- This same test failed, when parsed with basic llamaParse method ```LlamaParse(result_type="text")```

In [85]:
# negative test case
handler = agent.run(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=15"
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=20"
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 mit d=85",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: German. I need to use a tool to help me answer the question for each specified diameter.
Action: query_tool
Action Input: {'input': 'Zylinderrollenlager DIN 5412 mit d=15'}
Observation: For Zylinderrollenlager DIN 5412 with \( d = 15 \), the series includes:

- NU 1003: \( D = 35 \), \( B = 10 \), \( r_{\text{min}} = 0.6 \)
- N 1003: \( D = 35 \), \( B = 10 \), \( r_{\text{min}} = 0.6 \)

Thought: I need to continue using the tool to find the series for the other specified diameters.
Action: query_tool
Action Input: {'input': 'Zylinderrollenlager DIN 5412 mit d=20'}
Observation: For Zylinderrollenlager DIN 5412 with \( d = 20 \), the series includes:

- NU 1004: \( D = 42 \), \( B = 12 \), \( r_{\text{min}} = 0.6 \)
- N 1004: \( D = 42 \), \( B = 12 \), \( r_{\text{min}} = 0.6 \)

Thought: I need to continue using the tool to find the series for the last specified diameter.
Action: query_tool
Action Input: {'input': 'Zylinderrollenlager DIN

Test 4 -> Succeeded partially ✅
- d = 15 succeeded
- d = 20 succeeded
- d = 85 failed -> agent confused between d and D 

-> Agent needs stricter system prompting + the need for a table-lookup-tool 

#### slightly better user prompt (not system prompt)

In [ ]:
handler = agent.run(
    "Gib mir die ganze Reihe für den Zylinderrollenlager DIN5412 und Lagerreihe NU2 mit d=85. Achtung; Tabellenkopf 'd' and 'D' sind nicht gleich.",
    ctx=ctx)
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

async for ev in handler.stream_events():
    # if isinstance(ev, ToolCallResult):
    #     print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler